## 1. Import libraries used in the script

In [1]:
# Import libraries
# Make sure that you also have pip  and cvxopt installed in your environment before running the script

# Stock prices API, https://www.alphavantage.co/documentation/
from alpha_vantage.timeseries import TimeSeries
# To display data from Jupyter notebooks in Tableau, https://github.com/CFMTech/Jupytab
import jupytab
import pandas as pd
from time import sleep
# PyPortfolioOpt library for portfolio optimisation, https://pyportfolioopt.readthedocs.io
from pypfopt import risk_models, plotting, expected_returns, EfficientFrontier, EfficientSemivariance, DiscreteAllocation 

## 2. Get daily adjusted stock prices for the set list of tickers

In [2]:
# Define parameters for the API call
ts = TimeSeries(key='QFOETJ6MGPLUFGAR', output_format='pandas', indexing_type='date')
    
# Set the list of stocks to get data for. Start with the most recent stock to avoid getting Null values for stocks
# that were not trading at that time. 
equities = ['AMZN','GOOGL','CRM', 'TWTR','NVDA']
    
# Create an empty dataframe to store daily adjusted close prices for every stock.
# Use the sleep function to wait 12 seconds between API calls to comply
# with Alpha Vantage's policy of max 5 calls per minute.

daily_prices = pd.DataFrame()
for eq in equities:
    data, metadata = ts.get_daily_adjusted(eq, outputsize='full')
    data = pd.Series(data=data['5. adjusted close'], name=eq)
    if daily_prices.empty:
        daily_prices = data        
    else:
        daily_prices = pd.merge(daily_prices, data, on='date', how='inner')        
    print(f'Processed {eq}, waiting 12 seconds')
    sleep(12)
    
# Sort the dataframe by date, ascending order    
daily_prices.sort_index(inplace=True)
#daily_prices.tail(1)

# Output the data frame in a csv file
daily_prices.to_csv('daily_asset_prices.csv')

Processed AMZN, waiting 12 seconds
Processed GOOGL, waiting 12 seconds
Processed CRM, waiting 12 seconds
Processed TWTR, waiting 12 seconds
Processed NVDA, waiting 12 seconds


## 3. Find first and last dates and prices from the data

In [3]:
# Read from the csv
daily_prices = pd.read_csv('daily_asset_prices.csv', index_col='date')
daily_prices.head(5)

,AMZN,GOOGL,CRM,TWTR,NVDA
date,,,,,
2013-11-07,343.560,505.535540,54.35,44.90,13.675125
2013-11-08,350.310,509.588049,55.49,41.65,14.629422
2013-11-11,354.378,506.859627,56.12,42.90,14.751647
2013-11-12,349.530,507.456470,56.71,41.90,14.775152
2013-11-13,356.220,517.833502,57.21,42.60,15.184136


In [4]:
# Add new Index column, converting the 'date' index to a column
daily_prices_reset = daily_prices.reset_index()

In [5]:
# Get the prices on the first date in the data set
first_date = pd.DataFrame(daily_prices_reset.head(1))
first_date = first_date.melt(id_vars=['date'],var_name='Ticker', value_name='First price')
first_date = first_date.rename(columns={"date":"First date"})
    
# Get the prices on the last date in the data set
last_date = pd.DataFrame(daily_prices_reset.tail(1))
last_date = last_date.melt(id_vars=['date'],var_name='Ticker', value_name='Last price')
last_date = last_date.rename(columns={"date":"Last date"})

In [6]:
last_date

,Last date,Ticker,Last price
0,2021-05-27,AMZN,3230.11
1,2021-05-27,GOOGL,2362.68
2,2021-05-27,CRM,225.83
3,2021-05-27,TWTR,58.08
4,2021-05-27,NVDA,619.52


In [7]:
# Merge two data frames together on the Ticker column
minmax_prices = first_date.merge(last_date,how='left',on='Ticker')

In [8]:
# Calculate mean daily historical returns
# frequency: number of time periods in a year; 252 (the number of trading days in a year)
# https://pyportfolioopt.readthedocs.io/en/latest/ExpectedReturns.html
mu = expected_returns.mean_historical_return(daily_prices,
                                                returns_data=False,
                                                compounding=True,
                                                frequency=252)

mu

AMZN     0.346105
GOOGL    0.226923
CRM      0.207935
TWTR     0.034727
NVDA     0.658267
dtype: float64

In [9]:
# Covariance matrix
# https://pyportfolioopt.readthedocs.io/en/latest/RiskModels.html
# "Mean-variance optimisation (MVO) requires a good risk model, i.e a good estimator of covariance. 
# The sample covariance is the default choice, but often has coefficients with extreme errors which are
# particularly dangerous in MVO because the optimiser is likely to make large allocations based on these coefficients.
# One possible improvement is to move extreme values towards the centre, in a process called shrinkage." 
# (https://reasonabledeviations.com/notes/papers/ledoit_wolf_covariance/)
S = risk_models.CovarianceShrinkage(daily_prices,
                                    returns_data=False,
                                    frequency=252).ledoit_wolf()

S

,AMZN,GOOGL,CRM,TWTR,NVDA
AMZN,0.095069,0.049369,0.051362,0.051953,0.059612
GOOGL,0.049369,0.069187,0.047532,0.049786,0.056458
CRM,0.051362,0.047532,0.115453,0.056258,0.069606
TWTR,0.051953,0.049786,0.056258,0.293273,0.074274
NVDA,0.059612,0.056458,0.069606,0.074274,0.185219


In [10]:
# 10 year Treasury rate, as of 30 April 2021
# https://www.treasury.gov/resource-center/data-chart-center/interest-rates/pages/textview.aspx?data=yield
rate = 0.0165

# Inflation rate as of 13 April 2021
# https://tradingeconomics.com/united-states/inflation-cpi
inflation = 0.026

# Calculate the risk free rate (rfr) of borrowing/lending.
#https://www.investopedia.com/terms/r/risk-freerate.asp
rfr = rate - inflation 

## 4.1 Find the Max Sharpe portfolio

In [11]:
# Mean-variance portfolio

# Calculate Efficient Frontier
ef = EfficientFrontier(mu,S)

# Calculate weights for Mean-variance portfolio
weights_max_sharpe = ef.max_sharpe(risk_free_rate=rfr)
cleaned_weights_max_sharpe = ef.clean_weights()

cleaned_weights_max_sharpe

OrderedDict([('AMZN', 0.38128),
             ('GOOGL', 0.0),
             ('CRM', 0.0),
             ('TWTR', 0.0),
             ('NVDA', 0.61872)])

In [12]:
# Create a dataframe with weights that will be brought to Tableau
weights_max_sharpe = pd.DataFrame(cleaned_weights_max_sharpe, index=[0]).T.reset_index().rename(columns={"index":"Ticker",0:"Max Sharpe Weight"})

In [13]:
# Print portfolio summary and create new variables for each indicator
(expected_return, annual_volatility,sharpe_ratio) = ef.portfolio_performance(verbose=False)

#Create a dictionary with values and names of metrics, then convert it to a dataframe
summary = {'Expected annual return':expected_return,
            'Annual volatility':annual_volatility,
            'Sharpe Ratio':sharpe_ratio}

# Create a dataframe with portfolio summary that will be brought to Tableau
portfolio_summary_sharpe = pd.DataFrame.from_dict(summary,orient='index').reset_index().rename(columns={'index':'Metric',0:'Value'})

# Adding a column for the Method; True is to allow duplicates
portfolio_summary_sharpe.insert(0,'Method','Max Sharpe', True)

/home/anyalitica/anaconda3/envs/jupytab-notebook-env/lib/python3.7/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:407: UserWarning: The risk_free_rate provided to portfolio_performance is different to the one used by max_sharpe. Using the previous value.
  UserWarning,


## 4.2. Mean-variance portfolio allocation functions for Tableau

In [14]:
# Find prices on the last date in the dataframe

latest_price = daily_prices.iloc[-1]

#latest_price

# Set portfolio value, in USD. This value will be updated by a parameter in Tableau

portf_value = 10000

In [15]:
# Create a function to bring Leftover for Mean-variance portfolio to Tableau
# based on the portfolio value passed from Tableau

def leftover_sharpe_f(portf_value):
    allocation = DiscreteAllocation(cleaned_weights_max_sharpe,
                                latest_price,
                                total_portfolio_value=portf_value,
                                short_ratio=None)
    alloc, leftover_sharpe = allocation.lp_portfolio()
    #print(alloc)
    #print(leftover_sharpe)
    return float(leftover_sharpe)

In [16]:
# Testing the function
leftover_sharpe=leftover_sharpe_f(portf_value)

leftover_sharpe

574.6900000000005

In [17]:
# Create a function to bring share allocations for Mean-variance portfolio to Tableau
# based on the portfolio value passed from Tableau

# Calculate discrete allocation of shares (no fractional shares)

ticker = 'AMZN' # just to test the function, the complete list of tickers is passed from Tableau

def allocation_sharpe_f(portf_value,ticker):
    allocation = DiscreteAllocation(cleaned_weights_max_sharpe,
                                latest_price,
                                total_portfolio_value=portf_value,
                                short_ratio=None)
    alloc_sharpe, leftover_sharpe = allocation.lp_portfolio()
    # Check that the ticker exists in the dictionary 
    #(as some tickers might not have any shares allocated and won't appear in the output)
    if ticker in alloc_sharpe:
        shares_sharpe = alloc_sharpe[ticker]
    else:
        shares_sharpe = 0
    return int(shares_sharpe)

In [18]:
# Testing the function
shares_sharpe=allocation_sharpe_f(portf_value,ticker)

shares_sharpe

1

## 4.3 Cummulative return for the Mean-variance portfolio function for Tableau

In [19]:
# Create a function to calculate cummulative returns for the period based on the portfolio value set in Tableau,
# and bring the cummulative return value to Tableau
def cum_returns_sharpe_f(portf_value):
    
    allocation = DiscreteAllocation(cleaned_weights_max_sharpe,
                                latest_price,
                                total_portfolio_value=portf_value,
                                short_ratio=None)
    alloc_sharpe, leftover_sharpe = allocation.lp_portfolio()
    
    # Create copies of first_date_sharpe and last_date_sharpe dataframes
    first_date_sharpe = first_date.copy(deep=True)
    last_date_sharpe = last_date.copy(deep=True)
    
    # Add allocations to the first_date_sharpe dataframe
    first_date_sharpe['Allocation_sharpe']=first_date_sharpe['Ticker'].map(alloc_sharpe)
    
    # Multiply the number of shares by the first day price, and sum these values to find the total portfolio value
    # on the first day of the data set.
    first_date_sharpe['Portfolio cost'] = first_date_sharpe['First price']*first_date_sharpe['Allocation_sharpe']
    first_day_portf = first_date_sharpe.groupby(['First date']).sum()
    total_portf_first_day = first_day_portf.drop(['Allocation_sharpe', 'First price'],axis=1)
    
    # Add allocations to the last_date_sharpe dataframe
    last_date_sharpe['Allocation_sharpe']=last_date_sharpe['Ticker'].map(alloc_sharpe)
    #print(last_date_sharpe)
    
    # Multiply the number of shares by the last day price, and sum these values to find the total portfolio value
    # on the last day of the data set. 
    last_date_sharpe['Portfolio cost'] = last_date_sharpe['Last price']*last_date_sharpe['Allocation_sharpe']
    last_day_portf = last_date_sharpe.groupby(['Last date']).sum()
    total_portf_last_day = last_day_portf.drop(['Allocation_sharpe', 'Last price'],axis=1)
    
    # Append portfolio values on the first and last dates   
    total_portf = total_portf_first_day.append(total_portf_last_day, ignore_index=False)
    
    # Calculate cummulative returns for the Mean-variance portfolio
    cum_return_sharpe = (total_portf.iloc[-1] - total_portf.iloc[0])/total_portf.iloc[0]
    #cum_return_sharpe = cum_return_sharpe*100
    #print(cum_return_sharpe)
    #print(type(cum_return_sharpe))
    return float(cum_return_sharpe) 

In [19]:
# Testing the function
cum_returns_sharpe_f(10000)

18.623338165692665

## 5.1. Find the Mean-semivariance portfolio

In [20]:
# Mean-semivariance portfolio (risk is not important, maximising the ups)
# Calculate daily historical returns form daily prices
historical_returns = expected_returns.returns_from_prices(daily_prices)

# Calculate Efficient semivariance
# frequency: number of time periods in a year; 252 (the number of trading days in a year)
es = EfficientSemivariance(mu,historical_returns,frequency=252,verbose=False)

In [25]:
# Efficient_return takes the desired return of the resulting portfolio.
# If when running the code you get a Solver error, try lowering the return value below.
es.efficient_return(0.15)

# Calculate clean weights for the Mean-semivariance portfolio
weights_es = es.clean_weights()

In [26]:
# Create a dataframe with weights for the Mean-semivariance portfolio that will be brought to Tableau
weights_es_df = pd.DataFrame(weights_es, index=[0]).T.reset_index().rename(columns={"index":"Ticker",0:"Efficient semivariance Weight"})

In [23]:
# Print portfolio summary
# https://pyportfolioopt.readthedocs.io/en/latest/GeneralEfficientFrontier.html
# Create new variables for each indicator
(expected_return, semivariance,sortino_ratio) = es.portfolio_performance(verbose=False)

#Create a dictionary with values and names of metrics. Then convert it into a dataframe
summary_es = {'Expected annual return':expected_return,
                'Semivariance':semivariance,
                'Sortino Ratio':sortino_ratio}

# Create a dataframe with portfolio summary for the Mean-semivariance portfolio
#that will be brought to Tableau
portfolio_summary_es = pd.DataFrame.from_dict(summary_es,orient='index').reset_index().rename(columns={'index':'Metric',0:'Value'})

# Adding a column for the Method; True is to allow duplicates
portfolio_summary_es.insert(0,'Method','Efficient semivariance', True)

## 5.2. Mean-semivariance portfolio allocation functions for Tableau

In [24]:
# Create a function to bring the leftover value for Mean-semivariance portfolio to Tableau
# based on the portfolio value passed from Tableau

def leftover_es_f(portf_value):
    allocation_es = DiscreteAllocation(weights_es,
                                latest_price,
                                total_portfolio_value=portf_value,
                                short_ratio=None)
    alloc, leftover_es = allocation_es.lp_portfolio()   
    return float(leftover_es)

In [ ]:
# Testing the function

leftover_es=leftover_es_f(portf_value)

leftover_es

In [ ]:
# Create a function to bring share allocations for the Mean-semivariance portfolio to Tableau
# based on the portfolio value passed from Tableau

# Calculate discrete allocation of shares (no fractional shares)

ticker = 'AMZN' # just to test the function, the complete list of tickers is passed from Tableau

def allocation_es_f(portf_value,ticker):
    allocation_es = DiscreteAllocation(weights_es,
                                latest_price,
                                total_portfolio_value=portf_value,
                                short_ratio=None)
    alloc_es, leftover_es = allocation_es.lp_portfolio()
    # Check that the ticker exists in the dictionary 
    #(as some tickers might not have any shares allocated and won't appear in the output)
    if ticker in alloc_es:
        shares_es = alloc_es[ticker]
    else:
        shares_es = 0
    return int(shares_es)

In [ ]:
# Testing the function
shares_es=allocation_es_f(portf_value,ticker)

shares_es

## 5.3. Cummulative return for the Mean-semivariance portfolio function for Tableau

In [ ]:
# Create a function to calculate cummulative returns for the period based on the portfolio value set in Tableau,
# and bring the cummulative return value to Tableau
def cum_returns_es_f(portf_value):
    
    allocation_es = DiscreteAllocation(weights_es,
                                latest_price,
                                total_portfolio_value=portf_value,
                                short_ratio=None)
    alloc_es, leftover_es = allocation_es.lp_portfolio()
    
    # Create copies of first_date_sharpe and last_date_sharpe dataframes
    first_date_es = first_date.copy(deep=True)
    last_date_es = last_date.copy(deep=True)
    
    # Add allocations to the first_date_sharpe dataframe
    first_date_es['Allocation_es']=first_date_es['Ticker'].map(alloc_es)
    #print(first_date_es)
    
    # Multiply the number of shares by the first day price, and sum these values to find the total portfolio value
    # on the first day of the data set.
    first_date_es['Portfolio cost'] = first_date_es['First price']*first_date_es['Allocation_es']
    first_day_portf_es = first_date_es.groupby(['First date']).sum()
    total_portf_first_day_es = first_day_portf_es.drop(['Allocation_es', 'First price'],axis=1)
    
    # Add allocations to the last_date_sharpe dataframe
    last_date_es['Allocation_es']=last_date_es['Ticker'].map(alloc_es)
    #print(last_date_es)
    
    # Multiply the number of shares by the last day price, and sum these values to find the total portfolio value
    # on the last day of the data set.
    last_date_es['Portfolio cost'] = last_date_es['Last price']*last_date_es['Allocation_es']
    last_day_portf_es = last_date_es.groupby(['Last date']).sum()
    total_portf_last_day_es = last_day_portf_es.drop(['Allocation_es', 'Last price'],axis=1)
    
    # Append portfolio values on the first and last dates    
    total_portf_es = total_portf_first_day_es.append(total_portf_last_day_es, ignore_index=False)
    
    # Calculate cummulative returns for the Efficient semivariance portfolio
    cum_return_es = (total_portf_es.iloc[-1] - total_portf_es.iloc[0])/total_portf_es.iloc[0]
    #cum_return_es = cum_return_es*100
    #print(cum_return_sharpe)
    #print(type(cum_return_sharpe))
    return float(cum_return_es) 

In [ ]:
# Testing the function
cum_returns_es_f(10000)

## 6. Create final dataframes to pass to Tableau

In [ ]:
# Merge dataframes for both methods on the Ticker column
methods_combined_df = weights_max_sharpe.merge(weights_es_df,how='left',on='Ticker')
#print('This is methods_combined_df')
#print(methods_combined_df)
    
# Merge the methods_combined_df dataframe with the minmax_prices dataframe
final_combined_df = methods_combined_df.merge(minmax_prices,how='left',on='Ticker')
#print('This is final_combined_df')
#print(final_combined_df)
    
# Append dataframes with portfolio summary for both methods
portfolio_summary_combined = portfolio_summary_sharpe.append(portfolio_summary_es, ignore_index=True)

In [ ]:
# Create a tables connection for Jupitab server to bring dataframes to Tableau

tables = jupytab.Tables()
tables['combined_table'] = jupytab.DataFrameTable("Main table", dataframe=final_combined_df, include_index=True)
tables['portfolio_combined'] = jupytab.DataFrameTable("Portfolio summary", dataframe=portfolio_summary_combined, include_index=True)

In [ ]:
# Create a function connection for Jupytab Server to trigger functions in the script from Tableau

functions = jupytab.Functions()

functions['leftover_sharpe_f'] = jupytab.Function('Leftover Sharpe',leftover_sharpe_f)
functions['leftover_es_f'] = jupytab.Function('Leftover ES',leftover_es_f)
functions['allocation_es_f'] = jupytab.Function('Shares ES',allocation_es_f)
functions['allocation_sharpe_f'] = jupytab.Function('Shares Sharpe',allocation_sharpe_f)
functions['cum_returns_sharpe_f'] = jupytab.Function('Cummulative return Sharpe',cum_returns_sharpe_f)
functions['cum_returns_es_f'] = jupytab.Function('Cummulative return ES',cum_returns_es_f)

In [ ]:
# GET /schema
tables.render_schema()

In [ ]:
# GET /data
try:
    tables.render_data(REQUEST)
except NameError:
    print("Not available outside jupytab context")

In [ ]:
# POST /evaluate
try:
    functions.render_evaluate(REQUEST)
except NameError:
    print("Not available outside jupytab context")